# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

In [5]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [6]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [7]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [13]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]

In [14]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [15]:
len(words) # всего слов

173403

In [16]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [18]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [19]:
len(stopwords_ru)

151

In [21]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                       for sent in sents_tokenize ]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [22]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [23]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [25]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета. 

In [26]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [37]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [38]:
len(words) # всего слов

93069

In [39]:
len(set(words)) # уникальных слов

11153

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [40]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1481),
 ('весь', 1405),
 ('один', 659),
 ('раскольник', 567),
 ('свой', 549),
 ('сказать', 544),
 ('человек', 501),
 ('говорить', 498),
 ('пётр', 492),
 ('знать', 478),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('самый', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('дело', 363)]

In [41]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4793

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [46]:
#%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

(0, 0)

In [47]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [45]:
len(model.wv.vocab)

4793

In [48]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [52]:
# вектор слова
model.wv['старуха'][:10]

array([-0.0029392 , -0.00010589,  0.00229633,  0.00165925, -0.00215367,
       -0.00462498, -0.00481919, -0.00433117,  0.0011592 ,  0.00422687],
      dtype=float32)

In [53]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [58]:
# похожести слов 
model.wv.similarity('тварь', 'право')

-0.05190023

In [57]:
# самые похожие
model.wv.most_similar('топор')

[('немедленно', 0.4151383340358734),
 ('знаете', 0.3651875853538513),
 ('контора', 0.3563022017478943),
 ('казаться', 0.3269883692264557),
 ('давать', 0.2996198534965515),
 ('таки', 0.28955167531967163),
 ('желаля', 0.2892993986606598),
 ('грустный', 0.28776898980140686),
 ('утерпеть', 0.2860780358314514),
 ('сад', 0.28051406145095825)]

In [63]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

[('прислушиваться', 0.3774661421775818),
 ('занавеска', 0.33372658491134644),
 ('изложение', 0.32738548517227173),
 ('каторга', 0.326471745967865),
 ('мальчик', 0.3106190860271454),
 ('драка', 0.3016200065612793),
 ('тосковать', 0.3005722463130951),
 ('товарищ', 0.29852718114852905),
 ('владеть', 0.2978742718696594),
 ('дуня', 0.2952512800693512)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [64]:
model_path = "./our_w2v.model"
model.save(model_path)

In [65]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [66]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [67]:
# А теперь ваш код предобработки


In [68]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [69]:
len(sents_tokenize2)

254

Дополняем модель.

In [70]:
# И теперь ваш код обучения

(0, 0)

In [71]:
'ядро' in model.wv.vocab

False

In [72]:
'ядро' in our_model.wv.vocab

True

In [73]:
our_model.wv.most_similar('ядро')

[('проседь', 0.33910953998565674),
 ('содержимый', 0.33011293411254883),
 ('кусать', 0.32447952032089233),
 ('припомниться', 0.3141816556453705),
 ('радостный', 0.30989477038383484),
 ('пятнадцать', 0.309553325176239),
 ('хвататься', 0.3031322658061981),
 ('иерусалим', 0.3015998601913452),
 ('вес', 0.2926264703273773),
 ('сапог', 0.2899753451347351)]

Пример со старым словом.

In [74]:
our_model.wv.most_similar('сын')

[('внезапно', 0.3412628769874573),
 ('история', 0.3265072703361511),
 ('серьёзный', 0.322419673204422),
 ('стаканчик', 0.3128495216369629),
 ('новичок', 0.30967897176742554),
 ('четыре', 0.3076416850090027),
 ('неуклюжий', 0.3046972155570984),
 ('кирпич', 0.29859185218811035),
 ('поправиться', 0.2891137897968292),
 ('университетский', 0.28426387906074524)]

In [75]:
model.wv.most_similar('сын')

[('внезапно', 0.3412628769874573),
 ('история', 0.3265072703361511),
 ('серьёзный', 0.322419673204422),
 ('стаканчик', 0.3128495216369629),
 ('новичок', 0.30967897176742554),
 ('четыре', 0.3076416850090027),
 ('неуклюжий', 0.3046972155570984),
 ('кирпич', 0.29859185218811035),
 ('поправиться', 0.2891137897968292),
 ('университетский', 0.28426387906074524)]